# Capstone Project 
## Assignment 1 Part 1 
A Jupyter Note of the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from bs4 import BeautifulSoup # xml parser library

print('Libraries imported.')

Libraries imported.


Get the Wikipedia page and create a BeautifulSoup object in order to parse and find the relevant information

In [2]:
wiki_page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(wiki_page, 'lxml') # BeautifulSoup object

# print(soup) reveals the first table is the postcode table 
table = soup.find('table') 


Build a list with all the row except cells with a borough that is Not assigned 

In [3]:
df_list = []
for row in table.find_all('tr')[1:]:  # skip the first row, which contains column headings
    row_list = []
    for item in row.find_all('td'):
        row_list.append(item.text.replace('\n',''))
    if row_list[1] != 'Not assigned': # Ignore cells with a borough that is Not assigned 
        df_list.append(row_list)


Group multiple neighborhoods with the same postcode

In [4]:
from itertools import groupby

df_list_unique = []
for key, group in groupby(df_list, lambda x: [x[0],x[1]]):
    neighs = ''
    for item in group: 
        neighs = neighs + item[2] + ', ' # grouping multiple items
    key.append(neighs[:-2])
    df_list_unique.append(key)
df_list_unique[0:5] # just checking

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront, Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights, Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned']]

Now, build the pandas dataframe

In [5]:
column_names = ["PostalCode","Borough","Neighborhood"]

df = pd.DataFrame(df_list_unique, columns=column_names) # build the dataframe from the table

df.head(12) # checking again

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [6]:
df.shape # show the shape

(103, 3)

---
*Note to myself: instead of using itertool groupby function, dataframe groupby can also be use:  
df = df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x : ', '.join(x))*